In [5]:
# import pandas as pd
# import os
# from sqlalchemy import create_engine, text
# from dotenv import load_dotenv
# import psycopg2
# from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# # Load environment variables
# load_dotenv()

# DB_NAME = os.getenv("DB_NAME")
# DB_USER = os.getenv("DB_USER")
# DB_PASSWORD = os.getenv("DB_PASSWORD")
# DB_HOST = os.getenv("DB_HOST")
# DB_PORT = os.getenv("DB_PORT")

# # Step 1: Connect to default 'postgres' DB to create target DB if needed
# def create_database_if_not_exists():
#     conn = psycopg2.connect(
#         dbname="postgres",
#         user=DB_USER,
#         password=DB_PASSWORD,
#         host=DB_HOST,
#         port=DB_PORT
#     )
#     conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
#     cur = conn.cursor()

#     cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}'")
#     exists = cur.fetchone()
#     if not exists:
#         print(f"Creating database '{DB_NAME}'...")
#         cur.execute(f"CREATE DATABASE {DB_NAME}")
#     else:
#         print(f"Database '{DB_NAME}' already exists.")
    
#     cur.close()
#     conn.close()

# create_database_if_not_exists()

# # Step 2: Connect to the newly created/existing target database
# engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# # Step 3: Drop the table if it exists, then recreate it
# drop_and_create_table_sql = """
# DROP TABLE IF EXISTS inspections;

# CREATE TABLE inspections (
#     inspection_id BIGINT PRIMARY KEY,
#     dba_name VARCHAR(255),
#     aka_name VARCHAR(255),
#     license BIGINT,
#     facility_type VARCHAR(255),
#     risk VARCHAR(100),
#     address TEXT,
#     city VARCHAR(100),
#     state CHAR(2),
#     zip VARCHAR(10),
#     inspection_date DATE,
#     inspection_type VARCHAR(255),
#     results VARCHAR(100),
#     latitude DOUBLE PRECISION,
#     longitude DOUBLE PRECISION,
#     violations TEXT,
#     facility_type_clean VARCHAR(255),
#     violations_list TEXT,
#     violation_num_list TEXT
# );
# """

# with engine.connect() as conn:
#     for stmt in drop_and_create_table_sql.strip().split(";"):
#         if stmt.strip():
#             conn.execute(text(stmt))
#     print("Table 'inspections' dropped and recreated.")

# # Step 4: Load the CSV and insert data
# df = pd.read_csv("data/cleaned_chicago_data.csv")

# df["violations_list"] = df["violations_list"].astype(str)
# df["violation_num_list"] = df["violation_num_list"].astype(str)

# df.to_sql("inspections", engine, if_exists="append", index=False)
# print("Data loaded successfully into 'inspections'.")


In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# load .env, build engine…
load_dotenv()
engine = create_engine(
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

# 1) DROP & RECREATE
drop_sql = "DROP TABLE IF EXISTS inspections;"
create_sql = """
CREATE TABLE inspections (
    inspection_id BIGINT PRIMARY KEY,
    dba_name VARCHAR(255),
    aka_name VARCHAR(255),
    license_ BIGINT,
    facility_type VARCHAR(255),
    risk VARCHAR(100),
    address TEXT,
    city VARCHAR(100),
    state CHAR(2),
    zip VARCHAR(10),
    inspection_date DATE,
    inspection_type VARCHAR(255),
    results VARCHAR(100),
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    violations TEXT,
    facility_type_clean VARCHAR(255),
    violations_list TEXT[],
    violation_num_list INTEGER[]
);
"""

with engine.begin() as conn:
    conn.execute(text(drop_sql))
    conn.execute(text(create_sql))
    print("✔ Table 'inspections' dropped and recreated.")

# 2) Load your cleaned CSV
import ast
df = pd.read_csv("data/cleaned_chicago_data.csv")
df["violations_list"] = df["violations_list"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
df["violation_num_list"] = df["violation_num_list"].apply(
    lambda x: list(map(int, ast.literal_eval(x))) if pd.notna(x) and x != 'nan' else []
)

df.to_sql("inspections", engine, if_exists="append", index=False)
print("✔ Data loaded into 'inspections'.")

✔ Table 'inspections' dropped and recreated.


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "inspections_pkey"
DETAIL:  Key (inspection_id)=(2616252) already exists.

[SQL: INSERT INTO inspections (inspection_id, facility_id, inspection_date, inspection_type, results) VALUES (%(inspection_id__0)s, %(facility_id__0)s, %(inspection_date__0)s, %(inspection_type__0)s, %(results__0)s), (%(inspection_id__1)s, %(facility_id__1 ... 117201 characters truncated ... __999)s, %(facility_id__999)s, %(inspection_date__999)s, %(inspection_type__999)s, %(results__999)s)]
[parameters: {'inspection_id__0': 2616252, 'facility_id__0': 1, 'results__0': 'Pass', 'inspection_date__0': '2025-04-23', 'inspection_type__0': 'Canvass', 'inspection_id__1': 2616252, 'facility_id__1': 13922, 'results__1': 'Pass', 'inspection_date__1': '2025-04-23', 'inspection_type__1': 'Canvass', 'inspection_id__2': 2616253, 'facility_id__2': 2, 'results__2': 'No Entry', 'inspection_date__2': '2025-04-23', 'inspection_type__2':

In [3]:
import pandas as pd
from sqlalchemy import create_engine

DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

# Create the same engine as before
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Query some data
df2 = pd.read_sql("SELECT * FROM inspections;", engine)
print(df2)


        inspection_id                   dba_name                   aka_name  \
0             2616252  ROTI MODERN MEDITERRANEAN  ROTI MODERN MEDITERRANEAN   
1             2616253     MAGGIE GYROS & CHICKEN     MAGGIE GYROS & CHICKEN   
2             2616226    FORNO ROSSO ON RANDOLPH    FORNO ROSSO ON RANDOLPH   
3             2616171          TO KOREAN CUISINE          TO KOREAN CUISINE   
4             2616216                 PUP SOCIAL                 PUP SOCIAL   
...               ...                        ...                        ...   
264156          67848    SAFAH FOOD & LIQUOR INC    SAFAH FOOD & LIQUOR INC   
264157          67738     MICHAEL'S ON MAIN CAFE     MICHAEL'S ON MAIN CAFE   
264158          52234                   CAFE 608                   CAFE 608   
264159          67733                    TROQUET                    TROQUET   
264160         104236                 TEMPO CAFE                 TEMPO CAFE   

        license_  facility_type           risk     

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264161 entries, 0 to 264160
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   inspection_id        264161 non-null  int64  
 1   dba_name             264161 non-null  object 
 2   aka_name             264161 non-null  object 
 3   license_             264161 non-null  int64  
 4   facility_type        264161 non-null  object 
 5   risk                 264161 non-null  object 
 6   address              264161 non-null  object 
 7   city                 264161 non-null  object 
 8   state                264161 non-null  object 
 9   zip                  264161 non-null  int64  
 10  inspection_date      264161 non-null  object 
 11  inspection_type      264161 non-null  object 
 12  results              264161 non-null  object 
 13  latitude             264161 non-null  float64
 14  longitude            264161 non-null  float64
 15  violations       